In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device("cuda") 
device

device(type='cuda')

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# nltk.download('punkt')

import gensim.downloader as api
corpus = api.load('text8')

with open('data_set.csv', 'r', encoding='utf-8') as dh:
    list_of_words = []
    list_of_targets = []
    for i, line in enumerate(dh):
        if i > 0:
            line = line.strip()
            line = line.split('@')
            line[-1] = word_tokenize(line[-1].lower())
            list_of_words.append(line[-1])
            list_of_targets.append(line[1])
    dh.close()

emb_dim = 100

model = Word2Vec(corpus, min_count=1)
model.build_vocab(list_of_words, update=True)
model.train(list_of_words, total_examples=model.corpus_count, epochs=20)
model.save('word2vec_100d')

# model = Word2Vec.load('word2vec/word2vec_100d')

# Access embeddings
word_embeddings = model.wv
print(model.wv.most_similar("folly"))
print(word_embeddings['superstitious'])


[('beware', 0.6662392020225525), ('instincts', 0.6640303134918213), ('hrothgar', 0.6629712581634521), ('iambics', 0.6610066294670105), ('tenderness', 0.6592267155647278), ('treachery', 0.6587343811988831), ('pricking', 0.6584377288818359), ('handmaidens', 0.6532597541809082), ('syntheses', 0.6495845913887024), ('likeness', 0.6484972834587097)]
[-0.04048599  0.12905751  0.00644742 -0.159529    0.02644038 -0.04414839
  0.21951708  0.3806294  -0.11227641 -0.03641276 -0.4232945  -0.0071422
 -0.29037607  0.07959711  0.0026127  -0.18612812  0.10470393 -0.05000051
  0.14229856 -0.06897871  0.33387327  0.14040878 -0.2105612  -0.24505743
  0.04551506 -0.25178385  0.10444038 -0.29238918  0.06125954 -0.16147742
  0.06866395 -0.06218638  0.01277934 -0.25590542  0.3155617   0.21938728
 -0.07081258 -0.11284231 -0.01596291 -0.21802321 -0.01051729 -0.13837603
  0.23760548 -0.03235026  0.02319545  0.19486424 -0.28398556  0.10339832
  0.09314205  0.1400143  -0.04277408 -0.00628536 -0.19703516 -0.021182


In [31]:
word2vec_embeddings = []
for sentence in list_of_words:
    emb_sentence = np.empty((100,0))
    for word in sentence:
        emb_sentence = np.hstack((emb_sentence, np.reshape(model.wv[word], (100, 1))))
    word2vec_embeddings.append(torch.from_numpy(emb_sentence))

[tensor([[-3.9412, -0.9267, -2.7061,  ...,  1.0746,  0.4186,  0.0783],
         [-2.3593, -3.9707, -0.8063,  ...,  0.4888, -4.1320, -1.7129],
         [ 2.4661,  0.8048, -1.4633,  ..., -0.2462,  1.4758, -0.2967],
         ...,
         [-1.2573,  0.2858,  0.5560,  ..., -1.1954,  0.7159, -0.5242],
         [ 0.3853, -0.9891, -0.4556,  ..., -1.5132,  1.3594, -0.4834],
         [ 3.9091,  3.3672,  2.6747,  ...,  5.1283,  0.3862,  2.0992]],
        dtype=torch.float64),
 tensor([[-4.0200,  0.0496, -2.1398,  ...,  0.2156, -0.5186,  0.0783],
         [-4.4834, -1.7300, -2.6632,  ..., -3.2712, -0.1655, -1.7129],
         [ 2.0059,  1.2110,  1.7593,  ..., -0.2157,  0.9597, -0.2967],
         ...,
         [ 1.5051, -0.9950,  0.1927,  ..., -0.9618,  0.1586, -0.5242],
         [ 0.3136,  1.3651, -1.6010,  ...,  0.6988,  0.0302, -0.4834],
         [ 1.4273,  1.8177,  3.1454,  ...,  2.0422,  0.4781,  2.0992]],
        dtype=torch.float64),
 tensor([[ 6.0962e-01,  4.1855e-01, -8.3067e-01, -7.6488e-

In [32]:
import torchtext
import torchtext.vocab

# The first time you run this will download a ~823MB file
glove = torchtext.vocab.GloVe(name="6B", # trained on Wikipedia 2014 corpus
                              dim=100)   # embedding size = 100

c:\Python311\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Python311\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [152]:
# list_of_inc = []
# for i, sen in enumerate(list_of_words):
#     counter = 0
#     for j, wor in enumerate(sen):
#         if wor not in glove: 
#             counter += 1
#     if counter >= 1: list_of_inc.append((i, counter))
# print(len(list_of_inc))
# list_of_inc

85


[(10, 1),
 (116, 1),
 (158, 1),
 (230, 1),
 (295, 1),
 (343, 1),
 (381, 1),
 (418, 1),
 (486, 2),
 (512, 1),
 (554, 1),
 (586, 1),
 (599, 1),
 (600, 1),
 (609, 1),
 (652, 1),
 (666, 1),
 (696, 1),
 (733, 1),
 (747, 1),
 (755, 1),
 (766, 1),
 (781, 1),
 (785, 1),
 (797, 1),
 (832, 1),
 (860, 1),
 (886, 1),
 (887, 1),
 (890, 1),
 (945, 1),
 (1002, 1),
 (1012, 1),
 (1013, 1),
 (1029, 1),
 (1030, 1),
 (1042, 1),
 (1049, 1),
 (1067, 1),
 (1075, 1),
 (1089, 1),
 (1093, 1),
 (1101, 1),
 (1111, 3),
 (1163, 1),
 (1190, 1),
 (1199, 1),
 (1232, 1),
 (1239, 1),
 (1268, 1),
 (1346, 1),
 (1364, 1),
 (1365, 1),
 (1371, 1),
 (1384, 1),
 (1432, 1),
 (1477, 1),
 (1583, 1),
 (1691, 1),
 (1723, 1),
 (1776, 1),
 (1782, 1),
 (1785, 1),
 (1824, 1),
 (1846, 1),
 (1853, 1),
 (1864, 1),
 (1880, 1),
 (1893, 1),
 (1932, 1),
 (1941, 1),
 (1946, 2),
 (1977, 1),
 (2000, 1),
 (2095, 1),
 (2096, 1),
 (2105, 1),
 (2196, 1),
 (2206, 1),
 (2263, 1),
 (2271, 1),
 (2307, 1),
 (2327, 1),
 (2391, 1),
 (2457, 1)]

In [149]:
# abc = []
# for i, sen in enumerate(list_of_words):
#     for j, wor in enumerate(sen):
#             if wor not in glove: abc.append((list_of_targets[i],i, j, wor))

In [148]:
# miss_by_aut=[]
# i = 0
# inc = 0
# prev_aut = list_of_inc[0][0]
# for aut , _, _ in list_of_inc:
#     if aut == prev_aut:
#         inc += 1
#     else:
#         miss_by_aut.append([prev_aut, inc])
#         prev_aut = aut
#         i+=1
#         inc = 0
# miss_by_aut.append([aut, inc])
# miss_by_aut

ValueError: not enough values to unpack (expected 3, got 2)

In [37]:
glove_embeddings = []
for sentence in list_of_words:
    emb_sentence = torch.empty((100,0))
    for word in sentence:
        emb_sentence = torch.hstack((emb_sentence, torch.reshape(glove[word], (100, 1))))
    glove_embeddings.append(emb_sentence)

glove_embeddings

KeyboardInterrupt: 

In [35]:
a = glove['.']
a

tensor([-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598,
         0.4662, -0.0192,  0.4148, -0.3435,  0.2687,  0.0446,  0.4213, -0.4103,
         0.1546,  0.0222, -0.6465,  0.2526,  0.0431, -0.1945,  0.4652,  0.4565,
         0.6859,  0.0913,  0.2188, -0.7035,  0.1679, -0.3508, -0.1263,  0.6638,
        -0.2582,  0.0365, -0.1361,  0.4025,  0.1429,  0.3813, -0.1228, -0.4589,
        -0.2528, -0.3043, -0.1121, -0.2618, -0.2248, -0.4455,  0.2991, -0.8561,
        -0.1450, -0.4909,  0.0083, -0.1749,  0.2752,  1.4401, -0.2124, -2.8435,
        -0.2796, -0.4572,  1.6386,  0.7881, -0.5526,  0.6500,  0.0864,  0.3901,
         1.0632, -0.3538,  0.4833,  0.3460,  0.8417,  0.0987, -0.2421, -0.2705,
         0.0453, -0.4015,  0.1139,  0.0062,  0.0367,  0.0185, -1.0213, -0.2081,
         0.6407, -0.0688, -0.5864,  0.3348, -1.1432, -0.1148, -0.2509, -0.4591,
        -0.0968, -0.1795, -0.0634, -0.6741, -0.0689,  0.5360, -0.8777,  0.3180,
        -0.3924, -0.2339,  0.4730, -0.02